In [1]:
# tensorflow==2.3.1
# nltk==3.5
# numpy==1.18.5
# scikit_learn==0.23.2
# !pip install -q colorama==0.4.3
# !pip install -q Flask==1.1.2

In [13]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, CSVLogger
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from colorama import Fore, Style, Back
from keras.models import Sequential, load_model
from keras.utils import plot_model
from tensorflow import keras
import tensorflow as tf
import numpy as np
import colorama
import random
import pickle
import json
import os
import gc
colorama.init()

In [14]:
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
code_path = "C:\\Users\\user\\Python_Anaconda\\task\\06\\Code"
data_path = "C:\\Users\\user\\Python_Anaconda\\task\\06\\DataSet"
results_path = "C:\\Users\\user\\Python_Anaconda\\task\\06\\ChatbotResults"

json_file_path = os.path.join(data_path,'ChatbotData.json')
model_info_path = os.path.join(results_path,'model info')
images_path = os.path.join(results_path, "images")

In [16]:
def load_json_file(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    return data

In [17]:
def save_object(obj, filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

In [18]:
def load_object(filename,path):
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [19]:
data = load_json_file(json_file_path)
training_sentences, training_labels, labels, responses = [], [], [], []
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)

In [20]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [21]:
vocab_size = 10000
embedding_dim = 128
max_len = 200
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [22]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          1280000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 14)                238       
Total params: 1,282,574
Trainable params: 1,282,574
Non-trainable params: 0
_________________________________________________________________


In [23]:
model_name = 'chat_model'
plot_model(model, to_file= os.path.join(model_info_path, model_name+'.png'), show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [24]:
class GarbageCollectorCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

filepath = os.path.join(model_info_path, model_name+'.best.hdf5')
history_logger = CSVLogger(os.path.join(model_info_path,model_name+'History.csv'), separator=",", append=True)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
es = EarlyStopping(monitor='loss', patience=5)
callbacks_list = [GarbageCollectorCallback(), checkpoint, es, history_logger]

In [25]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, verbose=1, callbacks=callbacks_list)

Epoch 1/500
8/8 [==============================] - ETA: 0s - loss: 2.6299 - accuracy: 0.28 - ETA: 0s - loss: 2.6288 - accuracy: 0.28 - ETA: 0s - loss: 2.6281 - accuracy: 0.25 - ETA: 0s - loss: 2.6230 - accuracy: 0.25 - ETA: 0s - loss: 2.6214 - accuracy: 0.24 - ETA: 0s - loss: 2.6139 - accuracy: 0.28 - ETA: 0s - loss: 2.6123 - accuracy: 0.28 - ETA: 0s - loss: 2.6124 - accuracy: 0.27 - 1s 100ms/step - loss: 2.6124 - accuracy: 0.2775
Epoch 2/500
8/8 [==============================] - ETA: 0s - loss: 2.5898 - accuracy: 0.25 - ETA: 0s - loss: 2.5838 - accuracy: 0.28 - ETA: 0s - loss: 2.5777 - accuracy: 0.29 - ETA: 0s - loss: 2.5775 - accuracy: 0.28 - ETA: 0s - loss: 2.5777 - accuracy: 0.26 - ETA: 0s - loss: 2.5689 - accuracy: 0.27 - 1s 86ms/step - loss: 2.5689 - accuracy: 0.2775
Epoch 3/500
8/8 [==============================] - ETA: 0s - loss: 2.5801 - accuracy: 0.25 - ETA: 0s - loss: 2.5438 - accuracy: 0.29 - ETA: 0s - loss: 2.5386 - accuracy: 0.30 - ETA: 0s - loss: 2.5418 - accuracy: 0.2

8/8 [==============================] - ETA: 0s - loss: 2.2193 - accuracy: 0.34 - ETA: 0s - loss: 2.2414 - accuracy: 0.34 - ETA: 0s - loss: 2.4054 - accuracy: 0.27 - ETA: 0s - loss: 2.4145 - accuracy: 0.25 - ETA: 0s - loss: 2.4010 - accuracy: 0.25 - ETA: 0s - loss: 2.3672 - accuracy: 0.25 - ETA: 0s - loss: 2.3250 - accuracy: 0.27 - ETA: 0s - loss: 2.3194 - accuracy: 0.27 - 1s 100ms/step - loss: 2.3194 - accuracy: 0.2775
Epoch 24/500
8/8 [==============================] - ETA: 0s - loss: 2.4577 - accuracy: 0.15 - ETA: 0s - loss: 2.4481 - accuracy: 0.18 - ETA: 0s - loss: 2.3767 - accuracy: 0.21 - ETA: 0s - loss: 2.3475 - accuracy: 0.25 - ETA: 0s - loss: 2.3402 - accuracy: 0.26 - ETA: 0s - loss: 2.3015 - accuracy: 0.28 - ETA: 0s - loss: 2.3193 - accuracy: 0.27 - 1s 90ms/step - loss: 2.3163 - accuracy: 0.2775
Epoch 25/500
8/8 [==============================] - ETA: 0s - loss: 2.3030 - accuracy: 0.28 - ETA: 0s - loss: 2.3149 - accuracy: 0.28 - ETA: 0s - loss: 2.2763 - accuracy: 0.28 - ETA: 0

Epoch 45/500
8/8 [==============================] - ETA: 0s - loss: 2.2872 - accuracy: 0.25 - ETA: 0s - loss: 2.3665 - accuracy: 0.20 - ETA: 0s - loss: 2.3330 - accuracy: 0.22 - ETA: 0s - loss: 2.2524 - accuracy: 0.25 - ETA: 0s - loss: 2.3013 - accuracy: 0.24 - ETA: 0s - loss: 2.2367 - accuracy: 0.29 - ETA: 0s - loss: 2.2448 - accuracy: 0.27 - 1s 88ms/step - loss: 2.2479 - accuracy: 0.2775
Epoch 46/500
8/8 [==============================] - ETA: 0s - loss: 2.2108 - accuracy: 0.25 - ETA: 0s - loss: 2.1736 - accuracy: 0.31 - ETA: 0s - loss: 2.2184 - accuracy: 0.28 - ETA: 0s - loss: 2.2512 - accuracy: 0.27 - ETA: 0s - loss: 2.2331 - accuracy: 0.28 - ETA: 0s - loss: 2.2397 - accuracy: 0.27 - 1s 87ms/step - loss: 2.2397 - accuracy: 0.2775
Epoch 47/500
8/8 [==============================] - ETA: 0s - loss: 2.2629 - accuracy: 0.28 - ETA: 0s - loss: 2.1681 - accuracy: 0.29 - ETA: 0s - loss: 2.1777 - accuracy: 0.29 - ETA: 0s - loss: 2.1885 - accuracy: 0.29 - ETA: 0s - loss: 2.2389 - accuracy: 0

Epoch 69/500
8/8 [==============================] - ETA: 0s - loss: 1.6202 - accuracy: 0.40 - ETA: 0s - loss: 1.8827 - accuracy: 0.33 - ETA: 0s - loss: 1.9549 - accuracy: 0.30 - ETA: 0s - loss: 2.0187 - accuracy: 0.27 - 1s 73ms/step - loss: 2.0164 - accuracy: 0.2775
Epoch 70/500
8/8 [==============================] - ETA: 0s - loss: 1.9123 - accuracy: 0.28 - ETA: 0s - loss: 1.9883 - accuracy: 0.29 - ETA: 0s - loss: 2.0118 - accuracy: 0.28 - ETA: 0s - loss: 1.9880 - accuracy: 0.29 - ETA: 0s - loss: 2.0090 - accuracy: 0.28 - 1s 74ms/step - loss: 2.0113 - accuracy: 0.2775
Epoch 71/500
8/8 [==============================] - ETA: 0s - loss: 1.6615 - accuracy: 0.46 - ETA: 0s - loss: 2.0593 - accuracy: 0.29 - ETA: 0s - loss: 2.0348 - accuracy: 0.28 - ETA: 0s - loss: 2.0012 - accuracy: 0.27 - ETA: 0s - loss: 2.0013 - accuracy: 0.27 - 1s 76ms/step - loss: 2.0013 - accuracy: 0.2775
Epoch 72/500
8/8 [==============================] - ETA: 0s - loss: 2.2030 - accuracy: 0.18 - ETA: 0s - loss: 2.064

Epoch 97/500
8/8 [==============================] - ETA: 0s - loss: 1.6130 - accuracy: 0.50 - ETA: 0s - loss: 1.7411 - accuracy: 0.40 - ETA: 0s - loss: 1.7341 - accuracy: 0.41 - ETA: 0s - loss: 1.7394 - accuracy: 0.41 - ETA: 0s - loss: 1.7250 - accuracy: 0.41 - 1s 73ms/step - loss: 1.7250 - accuracy: 0.4141
Epoch 98/500
8/8 [==============================] - ETA: 0s - loss: 1.6288 - accuracy: 0.50 - ETA: 0s - loss: 1.5809 - accuracy: 0.47 - ETA: 0s - loss: 1.6243 - accuracy: 0.48 - ETA: 0s - loss: 1.7086 - accuracy: 0.44 - ETA: 0s - loss: 1.7240 - accuracy: 0.43 - 1s 80ms/step - loss: 1.7112 - accuracy: 0.4361
Epoch 99/500
8/8 [==============================] - ETA: 0s - loss: 1.5657 - accuracy: 0.50 - ETA: 0s - loss: 1.6484 - accuracy: 0.42 - ETA: 0s - loss: 1.7026 - accuracy: 0.42 - ETA: 0s - loss: 1.6953 - accuracy: 0.42 - ETA: 0s - loss: 1.6957 - accuracy: 0.42 - 1s 84ms/step - loss: 1.6994 - accuracy: 0.4229
Epoch 100/500
8/8 [==============================] - ETA: 0s - loss: 1.26

...

8/8 [==============================] - ETA: 0s - loss: 0.7391 - accuracy: 0.78 - ETA: 0s - loss: 0.7773 - accuracy: 0.75 - ETA: 0s - loss: 0.7612 - accuracy: 0.75 - ETA: 0s - loss: 0.8226 - accuracy: 0.69 - ETA: 0s - loss: 0.8173 - accuracy: 0.70 - 0s 51ms/step - loss: 0.8232 - accuracy: 0.7093
Epoch 171/500
8/8 [==============================] - ETA: 0s - loss: 0.8338 - accuracy: 0.62 - ETA: 0s - loss: 0.7731 - accuracy: 0.76 - ETA: 0s - loss: 0.7621 - accuracy: 0.76 - ETA: 0s - loss: 0.8184 - accuracy: 0.73 - 1s 65ms/step - loss: 0.8117 - accuracy: 0.7357
Epoch 172/500
8/8 [==============================] - ETA: 0s - loss: 0.6580 - accuracy: 0.81 - ETA: 0s - loss: 0.7838 - accuracy: 0.72 - ETA: 0s - loss: 0.7914 - accuracy: 0.71 - ETA: 0s - loss: 0.7986 - accuracy: 0.71 - ETA: 0s - loss: 0.8022 - accuracy: 0.71 - ETA: 0s - loss: 0.8023 - accuracy: 0.71 - 1s 84ms/step - loss: 0.8023 - accuracy: 0.7181
Epoch 173/500
8/8 [==============================] - ETA: 0s - loss: 0.6031 - accura

8/8 [==============================] - ETA: 0s - loss: 0.7540 - accuracy: 0.75 - ETA: 0s - loss: 0.7179 - accuracy: 0.80 - ETA: 0s - loss: 0.7091 - accuracy: 0.78 - ETA: 0s - loss: 0.7153 - accuracy: 0.79 - ETA: 0s - loss: 0.6742 - accuracy: 0.80 - 0s 59ms/step - loss: 0.6704 - accuracy: 0.8106
Epoch 196/500
8/8 [==============================] - ETA: 0s - loss: 0.6961 - accuracy: 0.71 - ETA: 0s - loss: 0.6815 - accuracy: 0.67 - ETA: 0s - loss: 0.7065 - accuracy: 0.66 - ETA: 0s - loss: 0.6819 - accuracy: 0.72 - ETA: 0s - loss: 0.6702 - accuracy: 0.72 - 0s 48ms/step - loss: 0.6740 - accuracy: 0.7225
Epoch 197/500
8/8 [==============================] - ETA: 0s - loss: 0.6494 - accuracy: 0.84 - ETA: 0s - loss: 0.6540 - accuracy: 0.80 - ETA: 0s - loss: 0.6554 - accuracy: 0.78 - ETA: 0s - loss: 0.6347 - accuracy: 0.78 - ETA: 0s - loss: 0.6445 - accuracy: 0.78 - 1s 80ms/step - loss: 0.6450 - accuracy: 0.7841
Epoch 198/500
8/8 [==============================] - ETA: 0s - loss: 0.7577 - accura

8/8 [==============================] - ETA: 0s - loss: 0.4141 - accuracy: 0.78 - ETA: 0s - loss: 0.4269 - accuracy: 0.82 - ETA: 0s - loss: 0.5238 - accuracy: 0.80 - ETA: 0s - loss: 0.5774 - accuracy: 0.80 - ETA: 0s - loss: 0.5556 - accuracy: 0.80 - ETA: 0s - loss: 0.5247 - accuracy: 0.82 - ETA: 0s - loss: 0.5456 - accuracy: 0.81 - ETA: 0s - loss: 0.5463 - accuracy: 0.81 - 1s 99ms/step - loss: 0.5463 - accuracy: 0.8106
Epoch 220/500
8/8 [==============================] - ETA: 0s - loss: 0.5322 - accuracy: 0.84 - ETA: 0s - loss: 0.4684 - accuracy: 0.92 - ETA: 0s - loss: 0.5232 - accuracy: 0.86 - ETA: 0s - loss: 0.5416 - accuracy: 0.84 - 0s 58ms/step - loss: 0.5460 - accuracy: 0.8414
Epoch 221/500
8/8 [==============================] - ETA: 0s - loss: 0.4465 - accuracy: 0.87 - ETA: 0s - loss: 0.4249 - accuracy: 0.92 - ETA: 0s - loss: 0.5065 - accuracy: 0.86 - ETA: 0s - loss: 0.5187 - accuracy: 0.84 - ETA: 0s - loss: 0.5168 - accuracy: 0.83 - ETA: 0s - loss: 0.5501 - accuracy: 0.81 - 0s 62

8/8 [==============================] - ETA: 0s - loss: 0.3537 - accuracy: 0.84 - ETA: 0s - loss: 0.4510 - accuracy: 0.83 - ETA: 0s - loss: 0.4684 - accuracy: 0.83 - ETA: 0s - loss: 0.4738 - accuracy: 0.83 - 1s 67ms/step - loss: 0.4732 - accuracy: 0.8370
Epoch 248/500
8/8 [==============================] - ETA: 0s - loss: 0.4313 - accuracy: 0.87 - ETA: 0s - loss: 0.4179 - accuracy: 0.85 - ETA: 0s - loss: 0.4613 - accuracy: 0.81 - ETA: 0s - loss: 0.4838 - accuracy: 0.81 - 0s 37ms/step - loss: 0.4838 - accuracy: 0.8106
Epoch 249/500
8/8 [==============================] - ETA: 0s - loss: 0.5084 - accuracy: 0.81 - ETA: 0s - loss: 0.5227 - accuracy: 0.81 - ETA: 0s - loss: 0.5229 - accuracy: 0.81 - ETA: 0s - loss: 0.5014 - accuracy: 0.83 - 0s 50ms/step - loss: 0.5002 - accuracy: 0.8370
Epoch 250/500
8/8 [==============================] - ETA: 0s - loss: 0.4084 - accuracy: 0.81 - ETA: 0s - loss: 0.5376 - accuracy: 0.75 - ETA: 0s - loss: 0.4915 - accuracy: 0.79 - ETA: 0s - loss: 0.4961 - accura

8/8 [==============================] - ETA: 0s - loss: 0.4384 - accuracy: 0.87 - ETA: 0s - loss: 0.4626 - accuracy: 0.82 - ETA: 0s - loss: 0.4629 - accuracy: 0.85 - ETA: 0s - loss: 0.4315 - accuracy: 0.86 - ETA: 0s - loss: 0.4110 - accuracy: 0.88 - ETA: 0s - loss: 0.4089 - accuracy: 0.88 - ETA: 0s - loss: 0.4206 - accuracy: 0.87 - 1s 96ms/step - loss: 0.4243 - accuracy: 0.8722
Epoch 273/500
8/8 [==============================] - ETA: 0s - loss: 0.5074 - accuracy: 0.84 - ETA: 0s - loss: 0.4491 - accuracy: 0.81 - ETA: 0s - loss: 0.4204 - accuracy: 0.83 - ETA: 0s - loss: 0.4082 - accuracy: 0.84 - ETA: 0s - loss: 0.4363 - accuracy: 0.85 - ETA: 0s - loss: 0.4164 - accuracy: 0.87 - 1s 74ms/step - loss: 0.4164 - accuracy: 0.8767
Epoch 274/500
8/8 [==============================] - ETA: 0s - loss: 0.4976 - accuracy: 0.81 - ETA: 0s - loss: 0.4150 - accuracy: 0.83 - ETA: 0s - loss: 0.4368 - accuracy: 0.85 - ETA: 0s - loss: 0.4059 - accuracy: 0.86 - ETA: 0s - loss: 0.4259 - accuracy: 0.86 - ETA: 

In [26]:
model.save(os.path.join(model_info_path, model_name+".h5"))
save_object(tokenizer, "tokenizer",results_path)
save_object(lbl_encoder, "label_encoder",results_path)

In [27]:
def chat():
    data = load_json_file(json_file_path)
    model_name = 'chat_model'
    model = load_model(filepath=os.path.join(model_info_path ,model_name+".h5"))
    tokenizer = load_object("tokenizer",results_path)
    lbl_encoder = load_object("label_encoder",results_path)
    max_len = 200
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]), 
                                                                          truncating='post', maxlen=max_len),verbose=0)
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))
print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: What are the risk factors for heart and blood vessel problems?
ChatBot: Stroke symptoms can include weakness or numbness on one side of the body, difficulty speaking, and vision problems.
ChatBot: Treatments for peripheral artery disease include lifestyle changes such as quitting smoking, exercise, and diet, and medication.
User: How can I manage my modifiable risk factors to reduce my risk of heart and blood vessel problems?
ChatBot: Long-term effects of heart and blood vessel problems can include heart attack, stroke, heart failure, and peripheral artery disease.
ChatBot: Risk factors include high blood pressure, smoking, obesity, and a diet high in saturated fats.
User: quit
